In [1]:
import glob
import pickle
import numpy
from music21 import converter, instrument, note, chord, stream
import tensorflow as tf
from tensorflow.keras import layers


In [2]:
tf.InteractiveSession()

In [3]:
file = "midi/original_metheny.mid"
midi = converter.parse(file)
parsedNotes = midi.flat.notes

#Printing only first 10 note objects
for element in parsedNotes[:10]:
    print(element, element.offset)

<music21.chord.Chord C4 A3 E4 G3 F2> 4.0
<music21.note.Note F> 4.0
<music21.note.Note E> 4.0
<music21.note.Note G#> 4.0
<music21.note.Note B> 4.0
<music21.note.Note G#> 4.25
<music21.note.Note G#> 4.5
<music21.note.Note G#> 4.75
<music21.note.Note G#> 5.0
<music21.note.Note E> 5.0


In [13]:
notes = []
for i,file in enumerate(glob.glob("midi/*.mid")):
    midi = converter.parse(file)
    print('\r', 'Parsing file', i, " ", file, end='')
    parsedNotes = None
    #print(parsedNotes)
    try: #check if file has instrument part
        print("*********************")
        print("If instrument part")
        s2 = instrument.partitionByInstrument(midi)
        print("s2:",s2)
        parsedNotes = s2.parts[0].recurse()
        print("After adding instrument part:",parsedNotes)
        print("**********************")
    except: #file has notes in flat structure
        print("Printing if notes part:",midi.flat.notes)
        parsedNotes = midi.flat.notes
        print("Printing after adding notes: ",parsedNotes)
        print("**********************")
        
    for element in parsedNotes:
        if isinstance(element, note.Note):
            notes.append(str(element.pitch))
        elif isinstance(element, chord.Chord):
            notes.append('.'.join(str(n) for n in element.normalOrder))
print("Notes:\n",notes[:20])
print("Lenght notes:\n",len(notes))

with open('notes', 'wb') as filepath:
    pickle.dump(notes, filepath)

 Parsing file 0   midi/original_metheny.mid*********************
If instrument part
s2: <music21.stream.Score 0x7f11908c91d0>
After adding instrument part: <music21.stream.iterator.RecursiveIterator for Part:Piano @:0>
**********************
Notes:
 ['4.5.7.9.0', 'G#2', 'G#2', 'G#2', 'G#2', 'G#2', 'G#2', 'G#2', '4.5.7.9.0', 'G#2', 'G#2', 'G#2', '9.10.2.5', 'C3', 'G#2', 'G#2', 'G#2', 'G#2', 'G#2', 'G#2']
Lenght notes:
 3265


In [24]:
#count the number of possible outputs of notes
n_vocab = (len(set(notes)))
print("n_vocab:",n_vocab)

seq_length = 100

#extract all pitch names
pitch_names = sorted(set(item for item in notes))
print("Sorted pitch_names, and length:\n",pitch_names,",",len(pitch_names))
print("*************************************")
#We will create a dictonary to map each pitch to an integer
note_to_int = dict((note,idx) for idx, note in enumerate(pitch_names))
print("Notes to int dictonary \n:",note_to_int)
print("*************************************")
network_input = []
network_output = []

#We will create input and output sequences below
for i in range(0, len(notes)-seq_length,1):
    sequence_in = notes[i:i+seq_length]
    sequence_out = notes[i+seq_length]
    #Mapping pitches in sequence_in to corresponding mapping in 'notes_to_int':
    network_input.append([note_to_int[char] for char in sequence_in])
    #Mapping output note:
    network_output.append(note_to_int[sequence_out])
#print("Sq_in:",sequence_in)
#print("Sq_in_len:",len(sequence_in))
#print("Sq_out:",sequence_out)
#print("Sq_out_len:",len(sequence_out))
print("Net_in:",network_input[:10])
print("Net_out:",network_output[:10])
print("\n")
n_pattern = len(network_input)
#reshaping input to an LSTM compatible format
network_input = numpy.reshape(network_input,(n_pattern, seq_length,1))
#normalizing input
network_input = network_input/float(n_vocab)
network_output = tf.keras.utils.to_categorical(network_output) 
print("Output to Categorical:\n", network_output.shape)


n_vocab: 143
Sorted pitch_names, and length:
 ['0', '0.1.3', '0.1.3.5.8', '0.1.5', '0.1.5.8', '0.2.4.7', '0.2.7', '0.3.5.8', '0.3.7', '0.4', '0.4.7', '1.4.6.9', '1.5.8', '10', '10.1.3.6', '10.11.3.6', '10.2.5', '11.0', '11.0.2.4.7', '11.0.4.7', '11.1.3.5.7', '11.2', '11.2.4.6.7', '11.2.4.7', '11.4', '2.3.5.7.10', '2.3.7', '2.3.7.10', '2.4.7.10', '2.5.7.10', '2.5.7.9.10', '2.5.8.10', '2.5.9', '2.6', '2.6.9', '2.7', '3.4.8', '3.4.8.10', '3.4.8.11', '3.5.7.11', '3.6.9.11.0', '3.7.10', '3.8', '4.5.7.11', '4.5.7.9.0', '4.5.7.9.11', '4.5.9.0', '4.6.8.11.1', '4.6.8.9', '4.6.9', '4.6.9.0', '4.7', '4.7.9', '4.7.9.0', '4.7.9.11.0', '4.8', '4.9', '5', '5.6.10.1', '5.7.10', '5.7.10.1', '5.7.9.11.1', '5.8.10.0.1', '5.9', '5.9.0', '6.7.11.2', '6.7.9.11.2', '6.9.11.2', '7', '7.10.0.2.3', '7.10.0.3', '7.10.1', '7.10.2', '7.11.2', '7.8', '7.8.0', '7.8.0.3', '7.8.10.0.3', '7.9', '7.9.10.1.3', '7.9.11.1.3', '7.9.11.2', '7.9.11.2.4', '7.9.2', '8.0.3', '8.1', '8.10', '8.9.1.4', '8.9.11.1.4', '9.0.2.4.5', '

In [25]:
len(network_input.reshape([-1,100])[1])
network_input.shape


(3165, 100, 1)

In [ ]:
def music_learner(network_input, n_vocab):
    '''Creating a neural net that'll take in the pitch matrix and
       try to learn the patter in it
    '''
    model = tf.keras.Sequential()
    model.add(layers.LSTM(512, input_shape = (network_input.shape[1], 
                                             network_input.shape[2]), return_sequences=True))
    model.add(layers.Dropout(0.3))
    model.add(layers.LSTM(512,return_sequences=True))
    model.add(layers.Dropout(0.3))
    model.add(layers.LSTM(512))
    model.add(layers.Dense(256))
    model.add(layers.Dropout(0.3))
    model.add(layers.Dense(n_vocab))
    model.add(layers.Activation('softmax'))
    model.compile(optimizer=tf.train.AdamOptimizer(0.001), loss='categorical_crossentropy')
    return model

model = music_learner(network_input, n_vocab)
model.summary()

'''We'll use model checkpoints to save weights after each epoch'''



filepath = 'weights.{epoch:02d}-{loss:.5f}.h5'
ckpt_callback = tf.keras.callbacks.ModelCheckpoint(filepath,monitor='loss',save_best_only=True, mode='min')

callback_list = [ckpt_callback]

'''We can also use previously saved weights and continue our model from a checkpoint'''

weights = ""
if(len(weights)>0): model.load_weights(weights)

'''Train the model'''
model.fit(network_input, network_output, epochs = 10, batch_size=64,
                                                      callbacks = callback_list)


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 100, 512)          1052672   
_________________________________________________________________
dropout (Dropout)            (None, 100, 512)          0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 100, 512)          2099200   
_________________________________________________________________
dropout_1 (Dropout)          (None, 100, 512)          0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 512)               2099200   
_________________________________________________________________
dense (Dense)                (None, 256)               131328    
_________________________________________________________________
dropout_2 (Dropout)          (None, 256)               0         
__________

In [20]:
#We will generate network input again
network_input = []
output = []
for i in range(0, len(notes) - seq_length, 1):
    sequence_in = notes[i:i + seq_length]
    sequence_out = notes[i + seq_length]
    network_input.append([note_to_int[char] for char in sequence_in])
    output.append(note_to_int[sequence_out])

n_patterns = len(network_input)
    
#Generate notes from the neural network based on a sequence of notes
#Randomly sample a sequence from the input as a starting point for the prediction
    
start = numpy.random.randint(0, len(network_input)-1)
int_to_note = dict((idx, note) for idx, note in enumerate(pitch_names))
pattern = network_input[start]
prediction_output = []

# generate 500 notes
for i,note_index in enumerate(range(500)):
    prediction_input = numpy.reshape(pattern, (1, len(pattern), 1))
    prediction_input = prediction_input / float(n_vocab)
    prediction = model.predict(prediction_input, verbose=0)
    index = numpy.argmax(prediction)
    result = int_to_note[index]
    print('\r', 'Predicted ', i, " ",result, end='')
    prediction_output.append(result)
    pattern.append(index)
    pattern = pattern[1:len(pattern)]
print(prediction_output)

NameError: name 'model' is not defined

In [ ]:
offset = 0
output_notes = []
# create note and chord objects based on the values generated by the model
for pattern in prediction_output:
    # pattern is a chord
    if ('.' in pattern) or pattern.isdigit():
        notes_in_chord = pattern.split('.')
        notes = []
        for current_note in notes_in_chord:
            new_note = note.Note(int(current_note))
            new_note.storedInstrument = instrument.Piano()
            notes.append(new_note)
        new_chord = chord.Chord(notes)
        new_chord.offset = offset
        output_notes.append(new_chord)
    # pattern is a note
    else:
        new_note = note.Note(pattern)
        new_note.offset = offset
        new_note.storedInstrument = instrument.Piano()
        output_notes.append(new_note)

    # increase offset each iteration so that notes do not stack
    offset += 0.5

midi_stream = stream.Stream(output_notes)
midi_stream.write('midi', fp='test_output.mid')

In [ ]:
print("a:","\n",+5)